**Bibliotecas Necessárias**

In [ ]:
from google.colab import files
import zipfile
import os

import glob
from PIL import Image

# !pip install facenet-pytorch
# from facenet_pytorch import MTCNN, InceptionResnetV1
# import torch
# from PIL import Image

# from sklearn.metrics import accuracy_score


In [ ]:
!pip install --upgrade ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.5 MB/s eta 0:00:00


In [ ]:
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2

**Extração de imagens**

In [ ]:
import os
import zipfile

zip_files = ['CelebA_subset.zip', 'LFW_subset.zip', 'UTKFace_subset.zip', 'WIDER_subset.zip']

for zip_file in zip_files:
    folder_name = zip_file.replace('.zip', '')
    os.makedirs(folder_name, exist_ok=True)
    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(folder_name)
        print(f"✅ Extraído: {zip_file}")
    except zipfile.BadZipFile:
        print(f"❌ Arquivo corrompido: {zip_file}")



✅ Extraído: CelebA_subset.zip
✅ Extraído: LFW_subset.zip
✅ Extraído: UTKFace_subset.zip
✅ Extraído: WIDER_subset.zip


**Bases de dados**

In [ ]:

base1_imgs = glob.glob('./CelebA_subset/**/*.jpg', recursive=True)
base2_imgs = glob.glob('./LFW_subset/**/*.jpg', recursive=True)
base3_imgs = glob.glob('./UTKFace_subset/**/*.jpg', recursive=True)
base4_imgs = glob.glob('./WIDER_subset/**/*.jpg', recursive=True)

# Junta tudo
all_imgs = base1_imgs + base2_imgs + base3_imgs + base4_imgs
print("Total de imagens:", len(all_imgs ))

Total de imagens: 20000


**YOLOv8 – Detecção**

In [ ]:
from ultralytics import YOLO

model_yolo = YOLO('yolov8n.pt')  # você pode usar yolov8s.pt, yolov8m.pt, etc.

def detectar_com_yolo(path_img):
    try:
        result = model_yolo(path_img)
        boxes = result[0].boxes
        return len(boxes) > 0
    except Exception as e:
        print(f"[YOLOv8] Erro em {path_img}: {e}")
        return False


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from facenet_pytorch import MTCNN
from PIL import Image
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(keep_all=False, device=device)

def detectar_com_mtcnn(path_img):
    try:
        img = Image.open(path_img).convert('RGB')
        face = mtcnn(img)
        return face is not None
    except Exception as e:
        print(f"[MTCNN] Erro em {path_img}: {e}")
        return False


In [ ]:
from facenet_pytorch import InceptionResnetV1

resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def detectar_com_facenet(path_img):
    try:
        img = Image.open(path_img).convert('RGB')
        face = mtcnn(img)
        if face is not None:
            face = face.unsqueeze(0).to(device)
            embedding = resnet(face)
            return embedding is not None
        else:
            return False
    except Exception as e:
        print(f"[FaceNet] Erro em {path_img}: {e}")
        return False


  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics import accuracy_score

def avaliar_modelo(imagens, funcao_modelo, nome_modelo):
    print(f"\n🔎 Avaliando modelo: {nome_modelo}")
    resultados = []
    for path in imagens:
        resultado = funcao_modelo(path)
        resultados.append(resultado)
    y_true = [1] * len(imagens)  # Supondo que todas as imagens contêm rosto
    acc = accuracy_score(y_true, resultados)
    print(f"✅ Acurácia total com {nome_modelo}: {acc:.2%} ({sum(resultados)}/{len(imagens)})")


In [ ]:
avaliar_modelo(all_imgs, detectar_com_yolo, "YOLOv8")
# avaliar_modelo(all_imgs, detectar_com_mtcnn, "MTCNN")
# avaliar_modelo(all_imgs, detectar_com_facenet, "FaceNet")


A saída de streaming foi truncada nas últimas 5000 linhas.
Speed: 5.3ms preprocess, 171.0ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /content/WIDER_subset/51_Dresses_wearingdress_51_180.jpg: 640x512 1 person, 190.6ms
Speed: 6.2ms preprocess, 190.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /content/WIDER_subset/2_Demonstration_Demonstrators_2_395.jpg: 480x640 4 persons, 182.2ms
Speed: 5.5ms preprocess, 182.2ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/WIDER_subset/18_Concerts_Concerts_18_429.jpg: 448x640 1 person, 192.9ms
Speed: 5.1ms preprocess, 192.9ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /content/WIDER_subset/12_Group_Group_12_Group_Group_12_138.jpg: 480x640 7 persons, 180.8ms
Speed: 6.4ms preprocess, 180.8ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/WIDER_subset/51_Dresses_wearingdress_51_4

In [ ]:
avaliar_modelo(all_imgs, detectar_com_mtcnn, "MTCNN")


🔎 Avaliando modelo: MTCNN
✅ Acurácia total com MTCNN: 98.30% (19659/20000)


In [ ]:
avaliar_modelo(all_imgs, detectar_com_facenet, "FaceNet")


🔎 Avaliando modelo: FaceNet
✅ Acurácia total com FaceNet: 98.30% (19659/20000)


In [ ]:

import os
import shutil
from PIL import Image

def salvar_imagens_falhas(imagens_falhas, nome_modelo):
    pasta_saida = f"falhas_{nome_modelo}"
    os.makedirs(pasta_saida, exist_ok=True)

    for i, path in enumerate(imagens_falhas):
        try:
            nome = os.path.basename(path)
            destino = os.path.join(pasta_saida, f"{i:04d}_{nome}")
            shutil.copy2(path, destino)
        except Exception as e:
            print(f"Erro ao copiar {path}: {e}")

    # Compactar em zip
    shutil.make_archive(pasta_saida, 'zip', pasta_saida)
    print(f"✅ Pasta '{pasta_saida}' criada e compactada como '{pasta_saida}.zip'")
